# 02. Solar energy yield estimation

Цель:
- загрузить здания с вычисленной площадью крыш
- оценить полезную площадь под фотоэлектрические панели
- рассчитать установленную мощность
- оценить годовую выработку электроэнергии


In [1]:
import geopandas as gpd
from pathlib import Path

In [3]:
PROJECT_ROOT = Path("/content/drive/MyDrive/solar_potential_project")
DATA_INTERIM = PROJECT_ROOT / "data" / "interim"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

DATA_PROCESSED.mkdir(exist_ok=True)

загрузка данных с площадью крыш

In [4]:
gdf = gpd.read_file(DATA_INTERIM / "estonia_tile_with_area.geojson")
gdf.head()

,height,confidence,area_m2,geometry
0,-1.0,-1.0,13.637127,"POLYGON ((569001.273 6531328.82, 569001.812 65..."
1,-1.0,-1.0,10.178461,"POLYGON ((560509.165 6534124.099, 560507.396 6..."
2,-1.0,-1.0,20.426216,"POLYGON ((573716.127 6529507.201, 573711.801 6..."
3,-1.0,-1.0,97.047615,"POLYGON ((572239.962 6528255.826, 572231.462 6..."
4,-1.0,-1.0,61.551992,"POLYGON ((575205.055 6530450.407, 575196.776 6..."


In [6]:
# Доля площади крыши, пригодная для установки панелей
USABLE_AREA_FACTOR = 0.6

# Удельная установленная мощность (кВт на м²)
POWER_DENSITY = 0.18

# Peak Sun Hours в Эстонии (кВт·ч / кВт / год)
PSH_YEAR = 950

# Коэффициент системных потерь (инвертор, загрязнение, температура)
SYSTEM_LOSS_FACTOR = 0.85

полезная площадь крыши
$$ A_{\text{usable}} = A \cdot k $$



In [7]:
gdf["usable_area_m2"] = gdf["area_m2"] * USABLE_AREA_FACTOR

установленная мощность
$$ P_{\text{installed}} = A_{\text{usable}} \cdot p_{\text{density}} $$

In [8]:
gdf["installed_power_kw"] = gdf["usable_area_m2"] * POWER_DENSITY

годовая выработка электроэнергии
$$ E_{\text{year}} = P_{\text{installed}} \cdot \text{PSH} \cdot \eta $$

In [9]:
gdf["annual_energy_kwh"] = (
    gdf["installed_power_kw"] * PSH_YEAR * SYSTEM_LOSS_FACTOR
)

In [10]:
gdf[[
    "area_m2",
    "usable_area_m2",
    "installed_power_kw",
    "annual_energy_kwh"
]].describe()

,area_m2,usable_area_m2,installed_power_kw,annual_energy_kwh
count,2132.000000,2132.000000,2132.000000,2132.000000
mean,78.173866,46.904320,8.442778,6817.542887
std,230.335725,138.201435,24.876258,20087.578557
min,3.463815,2.078289,0.374092,302.079270
25%,18.916592,11.349955,2.042992,1649.715987
50%,42.697945,25.618767,4.611378,3723.687786
75%,91.738530,55.043118,9.907761,8000.517183
max,8711.480192,5226.888115,940.839861,759728.187507


In [11]:
out = DATA_PROCESSED / "estonia_tile_with_energy.geojson"
gdf.to_file(out, driver="GeoJSON")

In [12]:
gdf.sample(min(500, len(gdf))).explore(
    column="annual_energy_kwh",
    cmap="YlOrRd",
    legend=True
)